In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import re

# Tải danh sách Stop Words tiếng Anh (chỉ cần chạy lần đầu)
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')

# 1. DỮ LIỆU ĐẦU VÀO (Corpus)
documents = [
    "Apple announces a new MacBook.",      # D1
    "Microsoft announces a new Surface.",   # D2
    "Apple and Microsoft are rivals."       # D3
]

# Danh sách Stop Words tiếng Anh chuẩn (lấy từ NLTK)
ENGLISH_STOP_WORDS = stopwords.words('english')

# Thêm Stop Words ngữ cảnh (các từ không phân biệt chủ đề)
CUSTOM_STOP_WORDS = ['announces', 'new', 'rivals', 'a', 'are', 'and']
FINAL_STOP_WORDS = set(ENGLISH_STOP_WORDS + CUSTOM_STOP_WORDS)


# 1. Hàm tiền xử lý: Chuẩn hóa, loại bỏ dấu câu và lọc Stop Words
def preprocess(doc):
    
    # 1.1. Chuẩn hóa về chữ thường (Đầu tiên!)
    doc = doc.lower()
    
    # 1.2. Loại bỏ Dấu câu/Ký tự đặc biệt
    # [^a-z\s] loại bỏ MỌI KÝ TỰ KHÔNG phải là chữ thường (a-z) hoặc khoảng trắng (\s).
    doc = re.sub(r'[^a-z\s]', '', doc) 
    
    # 1.3. Tách từ (Tokenize)
    tokens = doc.split()
    
    # 1.4. Lọc Stop Words
    return [word for word in tokens if word not in FINAL_STOP_WORDS]

# Thực hiện tiền xử lý
corpus = [preprocess(doc) for doc in documents]

# Số lượng tài liệu N
N = len(corpus) 

print("--- Dữ liệu sau tiền xử lý ---")
for i, doc in enumerate(corpus):
    print(f"D{i+1}: {doc} (Số từ: {len(doc)})")

# Tạo tập hợp các từ vựng duy nhất (Vocabulary)
vocabulary = sorted(list(set(word for doc in corpus for word in doc)))
print(f"\nTừ vựng (V): {vocabulary}")
print("-" * 50)

--- Dữ liệu sau tiền xử lý ---
D1: ['apple', 'macbook'] (Số từ: 2)
D2: ['microsoft', 'surface'] (Số từ: 2)
D3: ['apple', 'microsoft'] (Số từ: 2)

Từ vựng (V): ['apple', 'macbook', 'microsoft', 'surface']
--------------------------------------------------


# **Bước 2: Tính TF (Term Frequency)**

In [26]:
import pandas as pd

# 2. Hàm tính TF
def calculate_tf(corpus, vocabulary): # Định nghĩa hàm tính TF, nhận vào tập dữ liệu (corpus) và danh sách từ vựng.
    tf_matrix = {}
# Khởi tạo từ điển rỗng để lưu toàn bộ kết quả TF cho tất cả tài liệu.
    
    for i, doc in enumerate(corpus): # Tạo tên ID cho tài liệu hiện tại
        doc_id = f'D{i+1}'
        doc_length = len(doc)
        tf_scores = {}      # Khởi tạo từ điển rỗng để đếm số lần xuất hiện thô của từ trong tài liệu này.
        
        # Đếm tần suất
        for word in doc:
            tf_scores[word] = tf_scores.get(word, 0) + 1
            
        # Tính TF
        for word in vocabulary:
            count = tf_scores.get(word, 0)
            tf_matrix.setdefault(word, {})[doc_id] = count / doc_length # Tính TF và lưu vào ma trận
            
    return pd.DataFrame(tf_matrix).T.fillna(0) # Chuyển thành DataFrame để hiển thị đẹp hơn

tf_df = calculate_tf(corpus, vocabulary)        # Gọi hàm để tính TF và lưu kết quả vào biến tf_df.

print("--- MA TRẬN TF ---")
print(tf_df)
print("-" * 50)

--- MA TRẬN TF ---
            D1   D2   D3
apple      0.5  0.0  0.5
macbook    0.5  0.0  0.0
microsoft  0.0  0.5  0.5
surface    0.0  0.5  0.0
--------------------------------------------------


# **Bước 3: Tính IDF (Inverse Document Frequency)**

In [27]:
import numpy as np

# 3. Hàm tính IDF
def calculate_idf(corpus, vocabulary, N):
    df_scores = {} # Document Frequency: Số tài liệu chứa từ t
    
    # Tính DF
    for word in vocabulary:
        # Đếm số tài liệu chứa từ 'word'
        doc_count = sum(1 for doc in corpus if word in doc)
        df_scores[word] = doc_count

    # Tính IDF
    idf_scores = {}
    for word, df in df_scores.items():
        # Công thức IDF
        # np.log10(N / df)
        idf_scores[word] = np.log10(N / df)
        
    return pd.Series(idf_scores, name='IDF')

idf_series = calculate_idf(corpus, vocabulary, N)

print("--- VECTOR IDF ---")
print(idf_series)
print("-" * 50)

--- VECTOR IDF ---
apple        0.176091
macbook      0.477121
microsoft    0.176091
surface      0.477121
Name: IDF, dtype: float64
--------------------------------------------------


# **Bước 4: Tính TF-IDF và Ma trận Đặc trưng**
TF-IDF là tích của TF và IDF

In [28]:
# 4. Tính TF-IDF
# Nhân Ma trận TF với Vector IDF (dùng numpy broadcast)
tfidf_df = tf_df.multiply(idf_series, axis='index')

print("--- MA TRẬN TF-IDF (TRỌNG SỐ CUỐI CÙNG) ---")
# Làm tròn kết quả để dễ đọc
print(tfidf_df.round(4))
print("-" * 50)

# 5. PHÂN TÍCH KẾT QUẢ
print("--- PHÂN TÍCH KẾT QUẢ ---")

for doc_id in tfidf_df.columns:
    # 5.1. Tìm giá trị TF-IDF cao nhất
    max_score = tfidf_df[doc_id].max()
    
    # 5.2. Lọc ra TẤT CẢ các từ có giá trị bằng với giá trị cao nhất
    # Dùng Boolean indexing: tfidf_df[doc_id] == max_score
    top_words = tfidf_df[doc_id][tfidf_df[doc_id] == max_score].index.tolist()
    
    # 5.3. In kết quả
    print(f"Tài liệu {doc_id}:")
    print(f"  Trọng số cao nhất: {max_score:.4f}")
    print(f"  Các từ khóa phân biệt (bằng trọng số): {top_words}")
    print("-" * 25)

--- MA TRẬN TF-IDF (TRỌNG SỐ CUỐI CÙNG) ---
               D1      D2     D3
apple      0.0880  0.0000  0.088
macbook    0.2386  0.0000  0.000
microsoft  0.0000  0.0880  0.088
surface    0.0000  0.2386  0.000
--------------------------------------------------
--- PHÂN TÍCH KẾT QUẢ ---
Tài liệu D1:
  Trọng số cao nhất: 0.2386
  Các từ khóa phân biệt (bằng trọng số): ['macbook']
-------------------------
Tài liệu D2:
  Trọng số cao nhất: 0.2386
  Các từ khóa phân biệt (bằng trọng số): ['surface']
-------------------------
Tài liệu D3:
  Trọng số cao nhất: 0.0880
  Các từ khóa phân biệt (bằng trọng số): ['apple', 'microsoft']
-------------------------
